Para llevar a cabo el ejercicio de análisis de la prueba A/B del sistema de recomendaciones de la tienda en línea, aquí tienes un conjunto de actividades organizadas que puedes seguir:

## **Actividades para el Análisis de la Prueba A/B**

### **1. Definición de Objetivos del Estudio**
- **Objetivo Principal**: Evaluar el impacto de un nuevo embudo de pago en la conversión de usuarios mediante el análisis de eventos de marketing, registros y comportamientos de compra.
- **Metas Específicas**:
  - Determinar la efectividad del sistema de recomendaciones mejorado en la conversión de eventos (`product_page`, `product_card`, `purchase`).
  - Comparar la tasa de conversión entre los grupos de control (A) y de tratamiento (B).
  - Identificar si se cumplen los objetivos de al menos un 10% de aumento en cada etapa del embudo.

### **2. Exploración de Datos**
- **Carga de Datos**: Cargar los archivos CSV en un entorno de análisis (p. ej., Pandas en Python).
- **Conversión de Tipos**:
  - Revisar si las columnas de fechas están en el formato correcto y convertirlas si es necesario.
  - Asegurarse de que las columnas categóricas (como `event_name`, `group`) están en el tipo de datos adecuado.
- **Verificación de Valores Ausentes y Duplicados**:
  - Identificar y cuantificar valores ausentes en cada conjunto de datos.
  - Revisar duplicados y determinar cómo manejarlos (eliminación, imputación, etc.).

### **3. Análisis Exploratorio de Datos (EDA)**
- **Estudio de Conversión en el Embudo**:
  - Calcular las tasas de conversión en cada etapa del embudo para ambos grupos (A y B).
- **Distribución de Eventos por Usuario**:
  - Analizar el número total de eventos por usuario y comparar su distribución entre los grupos.
  - Identificar si hay usuarios en ambos grupos y cómo se distribuyen.
- **Distribución de Eventos por Día**:
  - Crear gráficos que muestren la cantidad de eventos a lo largo del tiempo para identificar tendencias.
- **Peculiaridades en los Datos**:
  - Revisar si hay patrones o anomalías en los datos que podrían afectar los resultados de la prueba A/B.

### **4. Evaluación de Resultados de la Prueba A/B**
- **Comparación de Resultados**:
  - Resumir las tasas de conversión finales de los grupos A y B.
  - Realizar una prueba Z para evaluar si hay diferencias estadísticamente significativas entre las proporciones de eventos en ambos grupos.
- **Interpretación de Resultados**:
  - Analizar si se alcanzaron los resultados esperados (al menos un 10% de mejora en cada etapa del embudo).
  - Reflexionar sobre la efectividad del nuevo sistema de recomendaciones.

### **5. Redacción de Conclusiones**
- **Resultados de la EDA**:
  - Describir las observaciones clave de la exploración de datos.
- **Conclusiones de la Prueba A/B**:
  - Resumir los hallazgos de la comparación de grupos y la significancia estadística.
  - Recomendar acciones basadas en los resultados (continuar, ajustar, o descartar el nuevo embudo).

### **6. Presentación de Resultados**
- Preparar un informe o presentación que incluya:
  - Metodología utilizada.
  - Resultados clave con visualizaciones (gráficos, tablas).
  - Conclusiones y recomendaciones.

Siguiendo estas actividades, podrás completar el análisis de la prueba A/B de manera estructurada y efectiva.

In [38]:
# Cargar todas las librerías
from scipy import stats as st
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np
import pandas as pd
import math as mt
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import re
import os
from plotly import graph_objects as go
from statsmodels.stats.proportion import proportions_ztest
#
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
# La función data_load() carga un archivo CSV desde una carpeta llamada 'datasets' y devuelve un DataFrame con los datos.

def data_load(file = 'ab_project_marketing_events_us.csv'):	
    ruta_archivo = os.path.join('datasets', file)
    data = pd.read_csv(
	    ruta_archivo
	)
    return data

In [3]:
# Cargar archivos CSV
marketing_events = data_load('ab_project_marketing_events_us.csv')
new_users = data_load('final_ab_new_users_upd_us.csv')
events = data_load('final_ab_events_upd_us.csv')
participants = data_load('final_ab_participants_upd_us.csv')

In [4]:
marketing_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


In [5]:
new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [6]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB


In [7]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB


In [8]:
# Convertir fechas
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])
new_users['first_date'] = pd.to_datetime(new_users['first_date'])
events['event_dt'] = pd.to_datetime(events['event_dt'])

In [9]:
marketing_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 580.0+ bytes


In [10]:
new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     58703 non-null  object        
 1   first_date  58703 non-null  datetime64[ns]
 2   region      58703 non-null  object        
 3   device      58703 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.8+ MB


In [11]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     423761 non-null  object        
 1   event_dt    423761 non-null  datetime64[ns]
 2   event_name  423761 non-null  object        
 3   details     60314 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 12.9+ MB


In [12]:
events['details'].describe()

count    60314.000000
mean        23.881219
std         72.228884
min          4.990000
25%          4.990000
50%          4.990000
75%          9.990000
max        499.990000
Name: details, dtype: float64

In [13]:
events['details'] = events['details'].fillna(events['details'].median())
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     423761 non-null  object        
 1   event_dt    423761 non-null  datetime64[ns]
 2   event_name  423761 non-null  object        
 3   details     423761 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 12.9+ MB


In [14]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB


In [15]:
participants.sample(3, random_state= 1)

,user_id,group,ab_test
3933,1E34D855ACE60AEF,A,interface_eu_test
3099,533875958A606F65,A,recommender_system_test
2674,51B3CC70709A2243,A,recommender_system_test


In [16]:
final_parts = participants[participants['ab_test'] == 'recommender_system_test']
final_parts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3675 entries, 0 to 3674
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  3675 non-null   object
 1   group    3675 non-null   object
 2   ab_test  3675 non-null   object
dtypes: object(3)
memory usage: 114.8+ KB


In [17]:
# Paso 1: Contar cuántos grupos tiene cada visitorId
group_count = final_parts.groupby('user_id')['group'].nunique().reset_index()

# Paso 2: Filtrar usuarios que están solo en un grupo
unique_visitors = group_count[group_count['group'] == 1]['user_id']

# Paso 3: Filtrar el DataFrame original para mantener solo esos usuarios
final_parts = final_parts[final_parts['user_id'].isin(unique_visitors)]

# Mostrando el DataFrame filtrado
print(final_parts.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3675 entries, 0 to 3674
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  3675 non-null   object
 1   group    3675 non-null   object
 2   ab_test  3675 non-null   object
dtypes: object(3)
memory usage: 114.8+ KB
None


In [18]:
# Valores nulos
print(f'Valores nulos:\n\nnew_users:')
print(new_users.isnull().sum())
print(f'\nevents:\n',events.isnull().sum())
print(f'\nfinal_participants:\n', final_parts.isnull().sum())

# Duplicados
print(f'\nDuplicados:\n')
print('new_users: ', new_users.duplicated().sum())
print('events: ', events.duplicated().sum())
print('final_participants: ', final_parts['user_id'].duplicated().sum())

Valores nulos:

new_users:
user_id       0
first_date    0
region        0
device        0
dtype: int64

events:
 user_id       0
event_dt      0
event_name    0
details       0
dtype: int64

final_participants:
 user_id    0
group      0
ab_test    0
dtype: int64

Duplicados:

new_users:  0
events:  0
final_participants:  0


In [19]:
dataset= final_parts.merge(events, on='user_id', how='inner').merge(new_users, on='user_id', how='inner')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23909 entries, 0 to 23908
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     23909 non-null  object        
 1   group       23909 non-null  object        
 2   ab_test     23909 non-null  object        
 3   event_dt    23909 non-null  datetime64[ns]
 4   event_name  23909 non-null  object        
 5   details     23909 non-null  float64       
 6   first_date  23909 non-null  datetime64[ns]
 7   region      23909 non-null  object        
 8   device      23909 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 1.6+ MB


In [20]:
embudo = dataset.groupby('event_name', as_index= False).agg({'user_id' : 'count'}).sort_values(by= 'user_id', ascending= False).reset_index(drop= True)
embudo.columns = ['event_name', 'users_cnt']
embudo.head()

,event_name,users_cnt
0,login,10837
1,product_page,6702
2,purchase,3210
3,product_cart,3160


In [21]:
# Filtrar los usuarios que tienen el evento 'product_cart'
usuarios_con_cart = dataset[dataset['event_name'] == 'product_cart']['user_id'].unique()

# Filtrar el dataframe original eliminando los usuarios que hicieron 'purchase' sin haber hecho 'product_cart'
dataset_filtrado = dataset[~((dataset['event_name'] == 'purchase') & (~dataset['user_id'].isin(usuarios_con_cart)))]

event_prd = dataset['event_dt'].dt.strftime('%Y-%m-%d')

dataset_filtrado['event_prd'] = event_prd

dataset_filtrado.head()

C:\Users\ulise\AppData\Local\Temp\ipykernel_24988\3800456308.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_filtrado['event_prd'] = event_prd


,user_id,group,ab_test,event_dt,event_name,details,first_date,region,device,event_prd
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,purchase,99.99,2020-12-07,EU,PC,2020-12-07
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:29,product_cart,4.99,2020-12-07,EU,PC,2020-12-07
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,product_page,4.99,2020-12-07,EU,PC,2020-12-07
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,login,4.99,2020-12-07,EU,PC,2020-12-07
4,A7A3664BD6242119,A,recommender_system_test,2020-12-20 15:46:06,product_page,4.99,2020-12-20,EU,iPhone,2020-12-20


In [22]:
# Ahora agrupa por 'event_name' para obtener el conteo de usuarios por evento
embudo = dataset_filtrado.groupby('event_name', as_index=False).agg({'user_id': 'count'}).sort_values(by='user_id', ascending=False).reset_index(drop=True)
embudo.columns = ['event_name', 'users_cnt']

# Ver el resultado
print(dataset_filtrado['user_id'].nunique())
print(embudo)

3674
     event_name  users_cnt
0         login      10837
1  product_page       6702
2  product_cart       3160
3      purchase        975


In [23]:
fig = go.Figure(go.Funnel(
    y = embudo['event_name'],
    x = embudo['users_cnt']
    ))
fig.show()

In [24]:
data_A = dataset_filtrado[dataset_filtrado['group'] == 'A']
data_A.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16857 entries, 0 to 23887
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     16857 non-null  object        
 1   group       16857 non-null  object        
 2   ab_test     16857 non-null  object        
 3   event_dt    16857 non-null  datetime64[ns]
 4   event_name  16857 non-null  object        
 5   details     16857 non-null  float64       
 6   first_date  16857 non-null  datetime64[ns]
 7   region      16857 non-null  object        
 8   device      16857 non-null  object        
 9   event_prd   16857 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(7)
memory usage: 1.4+ MB


In [25]:
data_B = dataset_filtrado[dataset_filtrado['group'] == 'B']
data_B.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4817 entries, 57 to 23908
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     4817 non-null   object        
 1   group       4817 non-null   object        
 2   ab_test     4817 non-null   object        
 3   event_dt    4817 non-null   datetime64[ns]
 4   event_name  4817 non-null   object        
 5   details     4817 non-null   float64       
 6   first_date  4817 non-null   datetime64[ns]
 7   region      4817 non-null   object        
 8   device      4817 non-null   object        
 9   event_prd   4817 non-null   object        
dtypes: datetime64[ns](2), float64(1), object(7)
memory usage: 414.0+ KB


In [26]:
# Ahora agrupa por 'event_name' para obtener el conteo de usuarios por evento
embudoA = data_A.groupby('event_name', as_index=False).agg({'user_id': 'count'}).sort_values(by='user_id', ascending=False).reset_index(drop=True)
embudoA.columns = ['event_name', 'users_cnt']

embudoB = data_B.groupby('event_name', as_index=False).agg({'user_id': 'count'}).sort_values(by='user_id', ascending=False).reset_index(drop=True)
embudoB.columns = ['event_name', 'users_cnt']

# Ver el resultado
print(embudoA)
print(data_A['user_id'].nunique())
print(embudoB)
print(data_B['user_id'].nunique())

     event_name  users_cnt
0         login       8272
1  product_page       5328
2  product_cart       2482
3      purchase        775
2747
     event_name  users_cnt
0         login       2565
1  product_page       1374
2  product_cart        678
3      purchase        200
927


In [27]:
fig = go.Figure(go.Funnel(
    y = embudoA['event_name'],
    x = embudoA['users_cnt']
    ))
fig.show()

In [28]:
fig = go.Figure(go.Funnel(
    y = embudoB['event_name'],
    x = embudoB['users_cnt']
    ))
fig.show()

In [29]:
data_dt = dataset_filtrado.groupby('event_prd', as_index= False)['user_id'].count()
print(len(data_dt))
print(data_dt.head())

# data_dt = dataset_filtrado.groupby('event_prd', as_index= False)['event_name'].count()
# print(len(data_dt))
# print(data_dt.head())

23
    event_prd  user_id
0  2020-12-07      632
1  2020-12-08      539
2  2020-12-09      676
3  2020-12-10      562
4  2020-12-11      497


In [40]:
fig = px.bar(data_dt, x='event_prd', y='user_id', title='Cantidad de eventos por día')
fig.update_xaxes(tickangle=45)
fig.show()

In [30]:
# # Filtrar los eventos relevantes
# conversion_events = events[events['event_name'].isin(['product_page', 'product_card', 'purchase'])]

# # Unir con la tabla de participantes para agregar el grupo
# conversion_events = conversion_events.merge(participants, on='user_id')

# # Contar usuarios únicos que realizaron cada evento por grupo
# conversion_rate = conversion_events.groupby(['group', 'event_name'])['user_id'].nunique().unstack()
# print(conversion_rate)



In [31]:
# Calcular la conversión en cada etapa
# conversion_rate['product_card'] = conversion_rate['product_card'] / conversion_rate['product_page']
# conversion_rate['purchase'] = conversion_rate['purchase'] / conversion_rate['product_card']

In [32]:
# # Crear una tabla pivot para organizar los eventos por usuario y grupo
# conversion_pivot = conversion_events.pivot_table(
#     index='group', 
#     columns='event_name', 
#     values='user_id', 
#     aggfunc='nunique'
# )

# # Ver las columnas y la tabla pivot generada
# print(conversion_pivot)

# # Calcular la conversión de product_page -> product_card -> purchase
# conversion_pivot['conversion_product_card'] = conversion_pivot['product_card'] / conversion_pivot['product_page']
# conversion_pivot['conversion_purchase'] = conversion_pivot['purchase'] / conversion_pivot['product_card']

# # Ver los resultados de la conversión
# print(conversion_pivot[['conversion_product_card', 'conversion_purchase']])

Las diferencias en las cantidades de usuarios entre los grupos A (2747) y B (927), en comparación con la cantidad esperada de 6,000 participantes, indican que la prueba A/B probablemente no se llevó a cabo como estaba planeada. A continuación se detallan algunas conclusiones y recomendaciones:

1. **Distribución desigual entre grupos**
   - La gran diferencia en el número de usuarios en los grupos A y B sugiere que el muestreo no fue equilibrado. Idealmente, en una prueba A/B, los grupos deben ser de tamaños similares para reducir el sesgo y asegurar que cualquier diferencia en las tasas de conversión se deba realmente a la variación introducida y no a un desequilibrio en el tamaño de las muestras.

2. **Tamaño de muestra insuficiente**
   - Con solo 3,674 participantes en total (A: 2,747, B: 927), estás lejos de la meta inicial de 6,000. Este tamaño de muestra reducido limita el poder estadístico de la prueba, es decir, la capacidad de detectar efectos significativos, especialmente si la mejora en las tasas de conversión es sutil. Con menos participantes, es más difícil distinguir si los cambios observados en las métricas son el resultado de la variación esperada o simplemente de la aleatoriedad.

3. **Conclusión sobre la adecuación de la prueba**
   - La combinación de una muestra más pequeña de lo previsto y una distribución desigual sugiere que la prueba A/B puede no ser lo suficientemente confiable como para tomar decisiones definitivas sobre los efectos de la variación probada.

**Recomendaciones**

1. **Repetir la prueba A/B**: Realizar una nueva prueba con un muestreo adecuado y alcanzar el tamaño de muestra planificado (6,000) o recalcular el tamaño de muestra necesario basado en los objetivos actuales. Asegúrate de que los grupos se distribuyan de manera uniforme.

2. **Asegurar una asignación balanceada**: Utiliza un mecanismo de aleatorización que garantice una asignación equitativa de los usuarios entre los grupos de prueba y control.

3. **Monitoreo de la recolección de datos**: Establece puntos de control para monitorear la distribución de los participantes en tiempo real, lo que permitirá detectar desequilibrios antes de que la prueba finalice y hacer los ajustes necesarios.

4. **Reevaluar el objetivo estadístico**: Si el objetivo es detectar una mejora del 10% en las tasas de conversión, asegura que el tamaño de muestra sea suficiente para esa sensibilidad específica. Un análisis de poder estadístico puede ayudarte a determinar cuántos participantes necesitas para una detección precisa de ese cambio.

Estas recomendaciones pueden ayudarte a obtener resultados más confiables y representativos para la próxima prueba A/B. ¿Te gustaría más detalle sobre alguna recomendación?

In [33]:
# Filtrar los datos para incluir solo eventos dentro de los 14 días posteriores a la inscripción
dataset['within_14_days'] = (dataset['event_dt'] - dataset['first_date']).dt.days <= 14
filtered_data = dataset[dataset['within_14_days']]

# Contar usuarios únicos en cada grupo y evento dentro de los 14 días
conversion_counts = filtered_data.groupby(['group', 'event_name'])['user_id'].nunique().unstack().fillna(0)

# Mostrar el resultado
conversion_counts

event_name,login,product_cart,product_page,purchase
group,,,,
A,2747,824,1780,872
B,927,255,523,256


In [34]:
# Calcular las tasas de conversión para cada grupo y evento
conversion_rates = pd.DataFrame()

# Conversión de product_page a product_cart y luego a purchase
conversion_rates['product_page'] = conversion_counts['product_page'] / conversion_counts['product_page']
conversion_rates['product_cart'] = conversion_counts['product_cart'] / conversion_counts['product_page']
conversion_rates['purchase'] = conversion_counts['purchase'] / conversion_counts['product_page']

# Mostrar las tasas de conversión
conversion_rates

,product_page,product_cart,purchase
group,,,
A,1.0,0.462921,0.489888
B,1.0,0.487572,0.489484


In [36]:
# Verificar los índices en conversion_counts
print(conversion_counts.index)

Index(['A', 'B'], dtype='object', name='group')


In [39]:
# Definir los conteos de éxitos y tamaños de muestra para cada evento
events = ['product_cart', 'purchase']
results = {}

for event in events:
    count_control = conversion_counts.loc['A', event]
    count_test = conversion_counts.loc['B', event]
    n_control = conversion_counts.loc['A', 'product_page']
    n_test = conversion_counts.loc['B', 'product_page']
    
    # Prueba Z para cada evento
    stat, p_value = proportions_ztest([count_control, count_test], [n_control, n_test])
    results[event] = {'z_stat': stat, 'p_value': p_value}

# Mostrar resultados de la prueba Z
results

{'product_cart': {'z_stat': -0.9931835856577905,
  'p_value': 0.32062049601759046},
 'purchase': {'z_stat': 0.016244280690050586, 'p_value': 0.9870395092337259}}

1. **Para el evento `product_cart`**:
   - **z_stat**: -0.993 (cerca de 0), lo que indica que no hay una gran desviación entre las tasas de conversión de los grupos A y B.
   - **p_value**: 0.321 (mayor que 0.05), lo que sugiere que la diferencia observada en la tasa de conversión entre los grupos no es estadísticamente significativa.
   
   Esto significa que no hay suficiente evidencia para concluir que existe una diferencia en la tasa de usuarios que agregan productos al carrito entre los grupos A y B.

2. **Para el evento `purchase`**:
   - **z_stat**: 0.016 (muy cercano a 0), lo que indica casi ninguna diferencia entre los grupos.
   - **p_value**: 0.987 (muy por encima de 0.05), indicando que no hay una diferencia estadísticamente significativa en la tasa de compra entre los grupos.

   Este resultado también sugiere que no hay una diferencia significativa en la tasa de conversión hacia compras entre los grupos A y B.

### Conclusión
Dado que ambos valores `p_value` son mayores que 0.05, no se observa una mejora significativa en ninguna de las etapas del embudo (`product_cart` y `purchase`) para el grupo de prueba en comparación con el grupo de control. Esto podría significar que la variación de prueba no está logrando el efecto esperado de mejora en la conversión en el embudo dentro de los 14 días posteriores a la inscripción.

## Conclusiones

1. **Resultados de la Prueba A/B**: Los resultados estadísticos muestran que no hay una diferencia significativa en las tasas de conversión entre los grupos A y B en las etapas del embudo analizadas (agregar al carrito y realizar compras). Esto indica que la variación introducida en la prueba no produjo el impacto esperado en la mejora de la conversión.

2. **Limitaciones en la Ejecución de la Prueba**:
   - **Desbalance en la distribución de grupos**: La prueba presenta una distribución desigual de participantes entre los grupos A (2,747) y B (927), lo que puede introducir sesgo en los resultados y reducir la confiabilidad de las conclusiones.
   - **Tamaño de muestra insuficiente**: La prueba se ejecutó con un total de 3,674 participantes, significativamente menos que los 6,000 esperados, lo cual limita el poder estadístico de la prueba. Con menos participantes, es más difícil detectar mejoras pequeñas en las tasas de conversión y diferenciar efectos reales de la variabilidad aleatoria.

3. **Recomendaciones**:
   - **Repetir la prueba con un tamaño de muestra adecuado**: Asegurarse de alcanzar el tamaño de muestra proyectado o ajustarlo en función del efecto deseado, para mejorar la precisión en la detección de cambios en las tasas de conversión.
   - **Balancear la asignación de usuarios**: Implementar un mecanismo de aleatorización que garantice una distribución equilibrada entre los grupos, lo que aumentará la validez de los resultados.
   - **Monitoreo durante la prueba**: Realizar un seguimiento de la asignación de participantes en tiempo real permitirá detectar y corregir cualquier sesgo o desbalance de inmediato.

En resumen, debido a la muestra insuficiente y al desbalance en los grupos, los resultados actuales no permiten tomar decisiones concluyentes sobre el efecto de la variación en las conversiones. Repetir la prueba con un diseño ajustado es necesario para obtener resultados confiables y representativos.